# Eat Safe, Love

## Part 1: Database and Jupyter Notebook Set Up

Import the data provided in the `establishments.json` file from your Terminal. Name the database `uk_food` and the collection `establishments`.

Within this markdown cell, copy the line of text you used to import the data from your Terminal. This way, future analysts will be able to repeat your process.

e.g.: Import the dataset with `YOUR IMPORT TEXT HERE`

mongoimport --db uk_food --collection establishments --file /Users/kathia/Desktop/ChallengesFolderCamp/NoSQL_databases/Resources/establishments.json --jsonArray

In [1]:
# Import dependencies
%pip install pymongo
from pymongo import MongoClient
from pprint import pprint

Note: you may need to restart the kernel to use updated packages.


In [2]:
# Create an instance of MongoClient
mongo = MongoClient(port=27017)

In [3]:
# confirm that our new database was created
print(mongo.list_database_names())

['admin', 'config', 'local']


In [4]:
# assign the uk_food database to a variable name
db = mongo['uk_food']

In [5]:
# review the collections in our new database
db = mongo['uk_food']
print(db.list_collection_names())

[]


In [6]:
# review a document in the establishments collection
establishments_collection = db['establishments']
sample_document = establishments_collection.find_one()
pprint(sample_document)

None


In [7]:
# assign the collection to a variable
establishments = db['establishments']

## Part 2: Update the Database

1. An exciting new halal restaurant just opened in Greenwich, but hasn't been rated yet. The magazine has asked you to include it in your analysis. Add the following restaurant "Penang Flavours" to the database.

In [8]:
# Create a dictionary for the new restaurant data
new_restaurant_data = {
    "BusinessName": "Penang Flavours",
    "BusinessType": "Restaurant/Cafe/Canteen",
    "BusinessTypeID": "",
    "AddressLine1": "Penang Flavours",
    "AddressLine2": "146A Plumstead Rd",
    "AddressLine3": "London",
    "AddressLine4": "",
    "PostCode": "SE18 7DY",
    "Phone": "",
    "LocalAuthorityCode": "511",
    "LocalAuthorityName": "Greenwich",
    "LocalAuthorityWebSite": "http://www.royalgreenwich.gov.uk",
    "LocalAuthorityEmailAddress": "health@royalgreenwich.gov.uk",
    "scores": {
        "Hygiene": "",
        "Structural": "",
        "ConfidenceInManagement": ""
    },
    "SchemeType": "FHRS",
    "geocode": {
        "longitude": "0.08384000",
        "latitude": "51.49014200"
    },
    "RightToReply": "",
    "Distance": 4623.9723280747176,
    "NewRatingPending": True
}


In [9]:
# Insert the new restaurant into the collection

insert_result = establishments_collection.insert_one(new_restaurant_data)

# Check if the insertion was successful
if insert_result.inserted_id:
    print("New restaurant 'Penang Flavours' was successfully inserted with ID:", insert_result.inserted_id)
else:
    print("Failed to insert the new restaurant.")


New restaurant 'Penang Flavours' was successfully inserted with ID: 651cb6701b5ec4d929e4353b


In [10]:
# Check that the new restaurant was inserted

newly_inserted_restaurant = establishments_collection.find_one({"BusinessName": "Penang Flavours"})

if newly_inserted_restaurant:
    print("New restaurant 'Penang Flavours' was found in the collection:")
    pprint(newly_inserted_restaurant)
else:
    print("New restaurant 'Penang Flavours' was not found in the collection.")


New restaurant 'Penang Flavours' was found in the collection:
{'AddressLine1': 'Penang Flavours',
 'AddressLine2': '146A Plumstead Rd',
 'AddressLine3': 'London',
 'AddressLine4': '',
 'BusinessName': 'Penang Flavours',
 'BusinessType': 'Restaurant/Cafe/Canteen',
 'BusinessTypeID': '',
 'Distance': 4623.972328074718,
 'LocalAuthorityCode': '511',
 'LocalAuthorityEmailAddress': 'health@royalgreenwich.gov.uk',
 'LocalAuthorityName': 'Greenwich',
 'LocalAuthorityWebSite': 'http://www.royalgreenwich.gov.uk',
 'NewRatingPending': True,
 'Phone': '',
 'PostCode': 'SE18 7DY',
 'RightToReply': '',
 'SchemeType': 'FHRS',
 '_id': ObjectId('651cb6701b5ec4d929e4353b'),
 'geocode': {'latitude': '51.49014200', 'longitude': '0.08384000'},
 'scores': {'ConfidenceInManagement': '', 'Hygiene': '', 'Structural': ''}}


2. Find the BusinessTypeID for "Restaurant/Cafe/Canteen" and return only the `BusinessTypeID` and `BusinessType` fields.

In [11]:
# Find the BusinessTypeID for "Restaurant/Cafe/Canteen" and return only the BusinessTypeID and BusinessType fields
query = {
    "BusinessType": "Restaurant/Cafe/Canteen"
}

projection = {
    "_id": 0,
    "BusinessTypeID": 1,
    "BusinessType": 1
}

result = establishments_collection.find_one(query, projection)
print(result)

{'BusinessType': 'Restaurant/Cafe/Canteen', 'BusinessTypeID': ''}


3. Update the new restaurant with the `BusinessTypeID` you found.

In [12]:
# Update the new restaurant with the correct BusinessTypeID
new_business_type_id = 1

update_query = {
    "BusinessName": "Penang Flavours"
}

update_data = {
    "$set": {
        "BusinessTypeID": new_business_type_id
    }
}

update_result = establishments_collection.update_one(update_query, update_data)

if update_result.modified_count > 0:
    print("The new restaurant's BusinessTypeID was updated successfully.")
else:
    print("No changes were made to the new restaurant's BusinessTypeID.")

The new restaurant's BusinessTypeID was updated successfully.


In [13]:
# Confirm that the new restaurant was updated
query_updated_restaurant = {
    "BusinessName": "Penang Flavours"
}

projection = {
    "_id": 0,  
    "BusinessTypeID": 1,
    "BusinessType": 1
}

updated_restaurant = establishments_collection.find_one(query_updated_restaurant, projection)

if updated_restaurant:
    print("Updated restaurant 'Penang Flavours' was found in the collection:")
    print(updated_restaurant)
    
    expected_business_type_id = 1 
    if updated_restaurant.get("BusinessTypeID") == expected_business_type_id:
        print("The BusinessTypeID of the updated restaurant is correct.")
    else:
        print("The BusinessTypeID of the updated restaurant does not match the expected value.")
else:
    print("Updated restaurant 'Penang Flavours' was not found in the collection.")


Updated restaurant 'Penang Flavours' was found in the collection:
{'BusinessType': 'Restaurant/Cafe/Canteen', 'BusinessTypeID': 1}
The BusinessTypeID of the updated restaurant is correct.


4. The magazine is not interested in any establishments in Dover, so check how many documents contain the Dover Local Authority. Then, remove any establishments within the Dover Local Authority from the database, and check the number of documents to ensure they were deleted.

In [14]:
# Find how many documents have LocalAuthorityName as "Dover"
dover_query = {
    "LocalAuthorityName": "Dover"
}

dover_count = establishments_collection.count_documents(dover_query)
print("Number of documents in Dover Local Authority:", dover_count)

Number of documents in Dover Local Authority: 0


In [15]:
# Delete all documents where LocalAuthorityName is "Dover"
delete_result = establishments_collection.delete_many(dover_query)


In [16]:
# Check if any remaining documents include Dover
remaining_count = establishments_collection.count_documents({})
print("Number of documents after deletion:", remaining_count)

if delete_result.deleted_count > 0:
    print(f"{delete_result.deleted_count} establishments in Dover were successfully deleted.")
else:
    print("No establishments in Dover were deleted.")


Number of documents after deletion: 1
No establishments in Dover were deleted.


In [17]:
# Check that other documents remain with 'find_one'
sample_document = establishments_collection.find_one()
print("Sample document after removal:")
print(sample_document)


Sample document after removal:
{'_id': ObjectId('651cb6701b5ec4d929e4353b'), 'BusinessName': 'Penang Flavours', 'BusinessType': 'Restaurant/Cafe/Canteen', 'BusinessTypeID': 1, 'AddressLine1': 'Penang Flavours', 'AddressLine2': '146A Plumstead Rd', 'AddressLine3': 'London', 'AddressLine4': '', 'PostCode': 'SE18 7DY', 'Phone': '', 'LocalAuthorityCode': '511', 'LocalAuthorityName': 'Greenwich', 'LocalAuthorityWebSite': 'http://www.royalgreenwich.gov.uk', 'LocalAuthorityEmailAddress': 'health@royalgreenwich.gov.uk', 'scores': {'Hygiene': '', 'Structural': '', 'ConfidenceInManagement': ''}, 'SchemeType': 'FHRS', 'geocode': {'longitude': '0.08384000', 'latitude': '51.49014200'}, 'RightToReply': '', 'Distance': 4623.972328074718, 'NewRatingPending': True}


5. Some of the number values are stored as strings, when they should be stored as numbers.

Use `update_many` to convert `latitude` and `longitude` to decimal numbers.

In [18]:
# Change the data type from String to Decimal for longitude and latitude
update_latitude_longitude = establishments_collection.update_many(
    {},
    [
        {"$set": {"geocode.longitude": {"$toDouble": "$geocode.longitude"}}},
        {"$set": {"geocode.latitude": {"$toDouble": "$geocode.latitude"}}}
    ]
)

print(f"Updated {update_latitude_longitude.modified_count} documents to store latitude and longitude as decimal numbers.")


Updated 1 documents to store latitude and longitude as decimal numbers.


Use `update_many` to convert `RatingValue` to integer numbers.

In [19]:
# Set non 1-5 Rating Values to Null
non_ratings = ["AwaitingInspection", "Awaiting Inspection", "AwaitingPublication", "Pass", "Exempt"]
establishments.update_many({"RatingValue": {"$in": non_ratings}}, [ {'$set':{ "RatingValue" : None}} ])

In [20]:
# Change the data type from String to Integer for RatingValue
establishments_collection.update_many(
    {"RatingValue": {"$exists": True}},
    [{"$set": {"RatingValue": {"$toInt": "$RatingValue"}}}]
)


In [21]:
# Check that the coordinates and rating value are now numbers
sample_document = establishments_collection.find_one()

if sample_document:
    print("Sample document after data type conversion:")
    print(sample_document)
else:
    print("No documents found in the collection.")


Sample document after data type conversion:
{'_id': ObjectId('651cb6701b5ec4d929e4353b'), 'BusinessName': 'Penang Flavours', 'BusinessType': 'Restaurant/Cafe/Canteen', 'BusinessTypeID': 1, 'AddressLine1': 'Penang Flavours', 'AddressLine2': '146A Plumstead Rd', 'AddressLine3': 'London', 'AddressLine4': '', 'PostCode': 'SE18 7DY', 'Phone': '', 'LocalAuthorityCode': '511', 'LocalAuthorityName': 'Greenwich', 'LocalAuthorityWebSite': 'http://www.royalgreenwich.gov.uk', 'LocalAuthorityEmailAddress': 'health@royalgreenwich.gov.uk', 'scores': {'Hygiene': '', 'Structural': '', 'ConfidenceInManagement': ''}, 'SchemeType': 'FHRS', 'geocode': {'longitude': 0.08384, 'latitude': 51.490142}, 'RightToReply': '', 'Distance': 4623.972328074718, 'NewRatingPending': True}
